<a href="https://colab.research.google.com/github/TsamayaDesigns/codeDivision-automation-with-python/blob/main/automation_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive & Set Working Directory:**

In [1]:
# 1️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

# 2️⃣ Set Working Directory
import os
working_dir = '/content/drive/MyDrive/codeDivision/automation_project'
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

# 3️⃣ Confirm Current Working Directory
print(f"Current working directory is: {os.getcwd()}")

Mounted at /content/drive/
Current working directory is: /content/drive/MyDrive/codeDivision/automation_project


**Query the Data:**

In [12]:
# import pandas as pd

from excel_tools import read_sheet
import re
from datetime import datetime
from smtp import send

fields = ['ID', 'Start_time', 'Completion_time', 'User_Email', 'Name', 'Last_modified_time', 'Full_Name', 'Need_365', 'Mac_or_PC', 'Moodle_Email', 'Attending', 'Alternative', 'Council', 'Postcode', 'NeuroDivergent', 'Woman', 'Minority', 'Disabled', 'Refugee']

# Create a class to represent the enrolment form
class EnrolmentForm:
  def __init__(self, filename, fields):
    self.filename = filename
    self.fields = fields
    self.submissions = self._load_data()

  def _load_data(self):
    return read_sheet(self.filename, fields = self.fields)

  def count(self):
      return len(self.submissions)

  def attendance_counts(self):
      attend_y = [s for s in self.submissions if str(s.Attending).lower() in ['yes', 'y']]
      attend_n = [s for s in self.submissions if str(s.Attending).lower() in ['no', 'n']]
      return len(attend_y), len(attend_n)

  def invalid_emails(self):
      EMAIL_REGEX = re.compile(
          r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
      )



  def __str__(self):
      return f"{self.filename}: {self.count()} submissions"

# Create form objects
filenames = ['data/EnrolmentForm1.xlsx', 'data/EnrolmentForm2.xlsx']
forms = [EnrolmentForm(filename, fields) for filename in filenames]

# Print form details
for form in forms:
  print(form)
  print('First two submissions:\n', form.submissions[:2])
  print()

# Attendance
for form in forms:
    attend_y, attend_n = form.attendance_counts()
    print(f"Attendance: ({form.filename}) - Yes: {attend_y}, No: {attend_n}")


data/EnrolmentForm1.xlsx: 17 submissions
First two submissions:
 [Row(ID=1, Start_time=datetime.datetime(2025, 7, 17, 17, 36, 32), Completion_time=datetime.datetime(2025, 7, 17, 17, 37, 33), User_Email='anonymous', Name=None, Last_modified_time=None, Full_Name='Piet Pompies', Need_365='Yes', Mac_or_PC='PC', Moodle_Email='piet@email.com', Attending='Yes', Alternative='NA', Council='Perth', Postcode='PH1', NeuroDivergent='No', Woman='No', Minority='No', Disabled='Yes', Refugee='No'), Row(ID=2, Start_time=datetime.datetime(2025, 7, 17, 17, 38, 34), Completion_time=datetime.datetime(2025, 7, 17, 17, 39, 15), User_Email='anonymous', Name=None, Last_modified_time=None, Full_Name='Jan Pampoen', Need_365='No', Mac_or_PC='Mac', Moodle_Email='jan@email.com', Attending='No', Alternative='NA', Council='Dundee', Postcode='DD1', NeuroDivergent='Yes', Woman='Yes', Minority='Yes', Disabled='No', Refugee='Yes')]

data/EnrolmentForm2.xlsx: 9 submissions
First two submissions:
 [Row(ID=1, Start_time=date

**Check Attendance:**

In [ ]:
# Check if person is attending
attend_y = []
attend_n = []

for form in forms:
    if form.Attending == 'Yes' or sub.Attending == 'yes' or sub.Attending == 'Y' or sub.Attending == 'y':
        attend_y.append(sub)
    else:
        attend_n.append(sub)

print(f'Attendance Yes: {len(attend_y)} \nAttendance No: {len(attend_n)}')

**Email address validation:**

In [ ]:
# Check if email address is valid
EMAIL_REGEX = re.compile(
    r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
)

def validate_emails(Moodle_Email):
  valid_emails = []
  invalid_emails = []

  if not EMAIL_REGEX.match(Moodle_Email.strip()):
    invalid_emails.append(f'Moodle_Email: {Moodle_Email.strip()}')
  else:
    valid_emails.append(f'Moodle_Email: {Moodle_Email.strip()}')

  return invalid_emails, valid_emails

# Collect all valid emails
all_valid_emails = []

# Print all valid emails
for sub in subs:
    invalid_emails, valid_emails = validate_emails(sub.Moodle_Email)

    if valid_emails:
      all_valid_emails.append((sub.full_name, sub.Moodle_Email))

print(f'Valid emails: {all_valid_emails}')

print()

# Print email address report (for each person)
print('Email address report:')
for sub in subs:
    invalid_emails, valid_emails = validate_emails(sub.Moodle_Email)

    if invalid_emails:
        print(sub.full_name, 'has invalid email(s):', invalid_emails)

    if valid_emails:
        print(sub.full_name, 'has valid email(s):', valid_emails)

**Office 365 Requirements:**

In [ ]:
# Establish Office 365 requirements
o365_y = []
o365_n = []

for sub in subs:
    if sub.Need_365 == 'Yes' or sub.Need_365 == 'yes' or sub.Need_365 == 'Y' or sub.Need_365 == 'y':
        o365_y.append(sub)
    else:
        o365_n.append(sub)

print(f'Require 365 - Yes: {len(o365_y)} \nRequire 365 - No: {len(o365_n)}')

**Available Dates:**

In [ ]:
# Check alternative dates for people unable attend

for sub in subs:
  if sub.Attending.strip().lower() in ['no', 'n']:
    try:
      date_str = sub.Alternative

      if isinstance(date_str, str):
        date = datetime.strptime(date_str.strip(), '%d/%m/%Y')
      elif isinstance(date_str, datetime):
        date = date_str
      else:
        raise ValueError("The Alternative date is not a string or datetime object.")

      print(f'{sub.full_name}, will be able to attend on: {date.strftime("%Y/%m/%d")}.')

    except ValueError:
      print(f'{sub.full_name}, entered an invalid date: \"{sub.Alternative.strip()}\", and can be contacted on {sub.Moodle_Email.strip()} to confirm alternative date(s).')





**Duplicate Email Check:**

In [ ]:
# Check for and display duplicate emails
duplicate_email = []
non_duplicate_email = []

# Read data from both forms
# subs_1 = read_sheet('EnrolmentForm1.xlsx', fields = fields)
# subs_2 = read_sheet('EnrolmentForm2.xlsx', fields = fields)

# Build a set of normalised emails from subs_1 for fast lookup
subs1_emails = {sub.Moodle_Email.strip().lower() for sub in subs_1}

# Iterate over subs_2 and check for duplicate emails
for sub2 in subs_2:
  subs2_email = sub2.Moodle_Email.strip().lower()
  if subs2_email in subs1_emails:
      duplicate_email.append((sub2.full_name, sub2.Moodle_Email))
  else:
    non_duplicate_email.append((sub2.full_name, sub2.Moodle_Email))

print(f'Duplicate emails in Form 2, that are already in Form 1: {duplicate_email}')
print(f'New (non-duplicate) emails in Form 2: {non_duplicate_email}')

**Export results to an Excel file:**

In [ ]:
# Create a DataFrame from the results
results_df = pd.DataFrame({
    'Duplicate Emails': pd.Series(duplicate_email),
    'Non-Duplicate Emails': pd.Series(non_duplicate_email)
})
print(results_df)
print()

# Write to Excel
results_df.to_excel('Email_Comparison_Results.xlsx', index=False)

print("Results exported to: 'Email_Comparison_Results.xlsx'")


**Sending Email:**

In [ ]:
# Prepare email body as a string
body_lines = []

body_lines.append('\nValid emails in Form 1:')
for name, email in all_valid_emails:
    body_lines.append(f'{name} - ({email})')

body_lines.append('\nNew Non-Duplicate emails in Form 2:')
for name, email in non_duplicate_email:
    body_lines.append(f'{name} - ({email})')

body_lines.append('\nDuplicate emails in Form 2, (already in Form 1):')
for name, email in duplicate_email:
    body_lines.append(f'{name} - ({email})')

email_body = '\n'.join(body_lines)

# Sending the Email
success = send(
    sender = "registrations@anysupport.co.uk",
    app_pass = "smss eolr jzjl pxvn",
    recip = "eugene@anysupport.co.uk",
    sub = "Test Email from Colab",
    body = email_body
)

print(email_body)